In [26]:
from bs4 import BeautifulSoup
from IPython.display import clear_output
import pandas as pd
import requests
import re
import json

## Project Luther

#### Scrape the recipes in Ecipurious.com

In [27]:
urls = []
search_page = 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page={}'

for url in range(1,474):
    formatted_search_page = search_page.format(url)
    urls.append(formatted_search_page)
urls

['https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=1',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=2',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=3',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=4',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=5',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=6',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=7',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=8',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=9',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=10',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=11',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=12',
 'https://www.epicurious.com/search/?meal=dinner%2Clunch%2Cbrunch&page=13',
 'https://www.epicuri

In [28]:
#trying to get the links for recipes
#url = 'https://www.epicurious.com/recipes/food/views/crispy-chicken-thighs-with-spring-vegetables#intcid=dt-recirc-cral2_1'
#response = requests.get(url)
#page = response.text
#soup = BeautifulSoup(page,'html.parser')
#soup.find('dd',class_='total-time').get_text().split(' ')[0]
#int(soup.find('div',class_='prepare-again-rating').find('span').get_text().split("%")[0])/100

#int(soup.find('span',{'reviews-count', 'reviewCount'}).get_text())
#soup.find(class_= 'pub-date').get_text()
#soup.find('span', {'itemprop': 'sodiumContent'}).get_text().split(' ')[0]

In [29]:
#crawl one search results page and return all recipes links
def Epicurious_crawlsearchpage(url):
    url_recipes = []
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'html.parser')
    for link in soup.find(class_='results-group').findAll(class_='photo-link'):
        if (link.attrs['href'].startswith('/recipes/food/views/')):
            url_recipes.append('https://www.epicurious.com'+link.attrs['href'])
            
    return url_recipes


In [30]:
def Epicurious_crawler(urls):
    count = 0
    url_recipes = []
    for url in urls:
        url_recipes.extend(Epicurious_crawlsearchpage(url))
        count +=1
        clear_output()
        print(count)
    return url_recipes
    
url_recipes = Epicurious_crawler(urls)
len(url_recipes)

473


7882

In [31]:
thefile = open('url_recipes.txt', 'w')
for item in url_recipes:
    thefile.write("%s\n" % item)

#### Save the features of the recipes to a dictionary

In [33]:

def Epicurious_crawlrecipe(urls_recipes):
    recipes_dict = {}
    count = 0
    for url in urls_recipes:

        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page,'html.parser')
        
        try:
            title = soup.find('h1', {'itemprop': 'name'}).get_text()
        except:
            title = 'notitle'
        try:
            make_it_again = int(soup.find('div',class_='prepare-again-rating').find('span').get_text().split("%")[0])/100
        except:
            make_it_again = 0
        try:
            rating = float(soup.find('span', class_="rating").get_text().split('/')[0])
        except:
            rating = 0
        try:
            reviews_count = int(soup.find('span',{'reviews-count', 'reviewCount'}).get_text())
        except:
            reviews_count = 0
        try:
            calories = soup.find('span', {'itemprop': 'calories'}).get_text()
        except:
            calories = 0
        try:
            carbohydrates = soup.find('span', {'itemprop': 'carbohydrateContent'}).get_text().split(' ')[0]
        except:
            carbohydrates = 0
        try:
            cholesterol = soup.find('span', {'itemprop': 'cholesterolContent'}).get_text().split(' ')[0]
        except:
            cholesterol = 0
        try:
            protein = soup.find('span', {'itemprop': 'proteinContent'}).get_text().split(' ')[0]
        except:
            protein = 0
        try:
            sodium = soup.find('span', {'itemprop': 'sodiumContent'}).get_text().split(' ')[0]
        except:
            sodium = 0       
        try:
            serving = soup.find(class_="per-serving").get_text().split('(')[1][0]
        except:
            serving = 0
        try:
            cooking_time = soup.find('dd',class_='total-time').get_text().split(' ')[0]
        except:
            cooking_time = 0
        try:
            date = soup.find(class_= 'pub-date').get_text()
        except:
            date = 0
        try:
            tags = [tag.get_text() for tag in soup.find(class_='tags').findAll('a') if ((tag.attrs['href']).startswith('/ingredient'))]
        except:
            tags = []

        recipes_dict[title] = {'url': url, 'rating': rating, 'serving': serving, 'calories': calories, 'carbohydrates': carbohydrates, 'cholesterol': cholesterol, 'protein': protein, 'make_it_again': make_it_again, 'reviews_count': reviews_count, 'cooking_time': cooking_time, 'date': date, 'sodium': sodium}

        for tag in tags:
            recipes_dict[title][tag] = 1
        
        count += 1
        clear_output()
        print(count)

    return recipes_dict

url_recipe = ['https://www.epicurious.com/recipes/food/views/crispy-chicken-thighs-with-kale-apricots-and-olives']


#recipes_dict = Epicurious_crawlrecipe(url_recipe)
#len(recipes_dict)
#recipes_dict



1


1

In [34]:
#recipes_dict

{'Crispy Chicken Thighs with Kale, Apricots, and Olives ': {'Apricot': 1,
  'Chicken': 1,
  'Garlic': 1,
  'Kale': 1,
  'Olive': 1,
  'Vinegar': 1,
  'White Wine': 1,
  'calories': '748',
  'carbohydrates': '33',
  'cholesterol': '219',
  'cooking_time': '45',
  'date': 'January 2018',
  'make_it_again': 1.0,
  'protein': '43',
  'rating': 3.0,
  'reviews_count': 4,
  'serving': '4',
  'sodium': '999',
  'url': 'https://www.epicurious.com/recipes/food/views/crispy-chicken-thighs-with-kale-apricots-and-olives'}}

In [47]:
urls0 = url_recipes[:1000]
urls1 = url_recipes[1001:2000]
urls2 = url_recipes[2001:3000]
urls3 = url_recipes[3001:4000]
urls4 = url_recipes[4001:5000]
urls5 = url_recipes[5001:6000]
urls6 = url_recipes[6001:7000]
urls7 = url_recipes[7001:8000]
urls8 = url_recipes[8001:]

count = 0 
#recipes_dict0 = Epicurious_crawlrecipe(urls0)
#recipes_dict1 = Epicurious_crawlrecipe(urls1)
#recipes_dict2 = Epicurious_crawlrecipe(urls2)
#recipes_dict3 = Epicurious_crawlrecipe(urls3)
#recipes_dict4 = Epicurious_crawlrecipe(urls4)
#recipes_dict5 = Epicurious_crawlrecipe(urls5)
recipes_dict6 = Epicurious_crawlrecipe(urls6)
recipes_dict7 = Epicurious_crawlrecipe(urls7)
recipes_dict8 = Epicurious_crawlrecipe(urls8)


881


In [52]:
recipes_dict = {}
recipes_dict.update(recipes_dict0)
recipes_dict.update(recipes_dict1)
recipes_dict.update(recipes_dict2)
recipes_dict.update(recipes_dict3)
recipes_dict.update(recipes_dict4)
recipes_dict.update(recipes_dict5)
recipes_dict.update(recipes_dict6)
recipes_dict.update(recipes_dict7)
recipes_dict.update(recipes_dict8)


recipes_dict

{'Coconut Shrimp Pancakes with Fresh Herbs ': {'Bacon': 1,
  'Bean': 1,
  'Chickpea': 1,
  'Egg': 1,
  'Green Onion/Scallion': 1,
  'Herb': 1,
  'Lime': 1,
  'Shrimp': 1,
  'calories': '563',
  'carbohydrates': '19',
  'cholesterol': '310',
  'cooking_time': '40',
  'date': 'January 2018',
  'make_it_again': 0.0,
  'protein': '32',
  'rating': 0.0,
  'reviews_count': 0,
  'serving': '4',
  'sodium': '998',
  'url': 'https://www.epicurious.com/recipes/food/views/coconut-shrimp-pancakes-with-fresh-herbs'},
 'Instant Pot Thai Coconut Clams ': {'Clam': 1,
  'Coconut': 1,
  'Seafood': 1,
  'Shellfish': 1,
  'calories': '343',
  'carbohydrates': '24',
  'cholesterol': '68',
  'cooking_time': '30',
  'date': 'January 2018',
  'make_it_again': 0.0,
  'protein': '36',
  'rating': 0.0,
  'reviews_count': 0,
  'serving': '4',
  'sodium': '2087',
  'url': 'https://www.epicurious.com/recipes/food/views/instant-pot-thai-coconut-clams-melissa-clark'},
 'Beef Short Ribs Satay (Satay Kra-Toog Ngua) ': 

In [53]:
with open('recipes_dict.json', 'w') as file:
    json.dump(recipes_dict, file)

In [51]:
recipes_dict_bkp = recipes_dict

#### Transform dictionary into a DataFrame

In [7]:
with open('recipes_dict', 'r') as file:
    recipes_dict = json.loads(file.read())


In [54]:
recipes_df = pd.DataFrame.from_dict(recipes_dict, orient='columns', dtype=None).T.fillna(0)
recipes_df

,Almond,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,Arugula,Asian Pear,Asparagus,...,cooking_time,date,make_it_again,protein,rating,reviews_count,serving,sodium,turkey,url
"""Antipasto"" Pasta with Sausage, Artichoke Hearts, and Sun-Dried Tomatoes",0,0,0,0,0,0,1,0,0,0,...,25,November 2017,0.00,26,0.0,0,4,974,0,https://www.epicurious.com/recipes/food/views/...
"""Bloody Mary"" Tomato Toast with Celery and Horseradish",0,0,0,0,0,0,0,0,0,0,...,15,July 2015,1.00,2,4.0,2,6,344,0,https://www.epicurious.com/recipes/food/views/...
"""Bow-Tie"" Pasta with Zucchini",0,0,0,0,0,0,0,0,0,0,...,0,July 1999,0.93,17,3.5,48,4,759,0,https://www.epicurious.com/recipes/food/views/...
"""Cromlet"" With Wilted Greens and Fennel and Olive Salad",0,0,0,0,0,0,0,0,0,0,...,0,February 2017,1.00,24,4.0,1,2,913,0,https://www.epicurious.com/recipes/food/views/...
"""Nextover"" Chicken Tacos with Quick Refried Beans",0,0,0,0,0,0,0,0,0,0,...,20,January 2017,0.67,65,3.0,3,2,1632,0,https://www.epicurious.com/recipes/food/views/...
"""No Nightmare"" Beef Wellington",0,0,0,0,0,0,0,0,0,0,...,0,August 2008,0.71,0,3.5,15,0,0,0,https://www.epicurious.com/recipes/food/views/...
"""Pizza"" Sandwiches",0,0,0,0,0,0,0,1,0,0,...,0,September 2009,1.00,34,3.0,5,4,1828,0,https://www.epicurious.com/recipes/food/views/...
"""Tomato Time"" Tart",0,0,0,0,0,0,0,0,0,0,...,0,August 2002,0.75,0,3.0,4,0,0,0,https://www.epicurious.com/recipes/food/views/...
"'Wichcraft's Roasted Turkey, Avocado, Bacon, Onion Relish, & Aïoli on Ciabatta",0,0,0,0,0,0,0,0,0,0,...,0,December 2006,1.00,55,4.0,20,4,1291,1,https://www.epicurious.com/recipes/food/views/...
10-Minute Chicken Flatbreads with Hummus and Yogurt,0,0,0,0,0,0,0,0,0,0,...,10,August 2017,0.83,35,3.5,6,4,917,0,https://www.epicurious.com/recipes/food/views/...


In [9]:
recipes_df['rating'].value_counts()

3.5    3066
3.0    1735
4.0    1251
0.0     490
2.5     481
2.0     174
1.0      54
1.5      39
Name: rating, dtype: int64

In [11]:
attributes = [x for x in recipes_df.columns]
attributes

['Almond',
 'Anchovy',
 'Anise',
 'Apple',
 'Apple Juice',
 'Apricot',
 'Artichoke',
 'Arugula',
 'Asian Pear',
 'Asparagus',
 'Avocado',
 'Bacon',
 'Banana',
 'Barley',
 'Basil',
 'Bass',
 'Bean',
 'Beef',
 'Beef Rib',
 'Beef Shank',
 'Beef Tenderloin',
 'Beer',
 'Beet',
 'Bell Pepper',
 'Berry',
 'Bitters',
 'Blackberry',
 'Blue Cheese',
 'Blueberry',
 'Bok Choy',
 'Bourbon',
 'Bran',
 'Brandy',
 'Breadcrumbs',
 'Brie',
 'Brisket',
 'Broccoli',
 'Broccoli Rabe',
 'Brown Rice',
 'Brussels Sprout',
 'Buffalo',
 'Bulgur',
 'Butter',
 'Buttermilk',
 'Butternut Squash',
 'Cabbage',
 'Calvados',
 'Cantaloupe',
 'Capers',
 'Caraway',
 'Cardamom',
 'Carrot',
 'Cashew',
 'Cauliflower',
 'Caviar',
 'Celery',
 'Champagne',
 'Chard',
 'Cheddar',
 'Cheese',
 'Cherry',
 'Chestnut',
 'Chicken',
 'Chickpea',
 'Chile Pepper',
 'Chive',
 'Chocolate',
 'Cilantro',
 'Cinnamon',
 'Citrus',
 'Clam',
 'Clove',
 'Coconut',
 'Cod',
 'Coffee',
 'Cognac/Armagnac',
 'Collard Greens',
 'Condiment',
 'Coriander',

In [13]:
recipes_df.corr()

,Almond,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,Arugula,Asian Pear,Asparagus,...,Whiskey,White Wine,Whole Wheat,Wild Rice,Wine,Yellow Squash,Yogurt,Zucchini,rating,turkey
Almond,1.000000,-0.008273,0.009936,0.015791,-0.002966,0.018133,-0.011820,0.006201,-0.003925,-0.015525,...,-0.004196,0.006498,-0.006121,-0.004196,-0.013503,-0.004692,0.041911,0.013503,0.019657,-0.021481
Anchovy,-0.008273,1.000000,0.030022,-0.010309,-0.001531,-0.005150,0.039449,-0.009178,-0.002026,0.026885,...,-0.002166,0.006303,-0.003159,-0.002166,-0.006970,-0.002422,-0.010632,0.035985,-0.012126,-0.011088
Anise,0.009936,0.030022,1.000000,-0.009795,0.093244,0.023420,-0.005798,-0.008720,-0.001925,-0.007614,...,-0.002058,-0.008657,-0.003002,-0.002058,0.035434,-0.002301,-0.010103,-0.008846,0.017451,0.002911
Apple,0.015791,-0.010309,-0.009795,1.000000,0.034361,0.021704,-0.014728,-0.015683,0.052659,-0.019344,...,-0.005229,-0.002443,0.029328,0.021690,-0.016825,-0.005846,0.013693,-0.022472,-0.022631,0.005657
Apple Juice,-0.002966,-0.001531,0.093244,0.034361,1.000000,-0.001847,-0.002188,-0.003291,-0.000726,-0.002873,...,-0.000777,-0.003267,-0.001133,-0.000777,-0.002499,-0.000868,-0.003812,-0.003338,0.015583,0.031542
Apricot,0.018133,-0.005150,0.023420,0.021704,-0.001847,1.000000,-0.007358,-0.011068,-0.002443,-0.009664,...,-0.002612,0.053042,-0.003810,0.050285,-0.008406,-0.002921,0.020325,-0.011227,0.000852,-0.013372
Artichoke,-0.011820,0.039449,-0.005798,-0.014728,-0.002188,-0.007358,1.000000,-0.002351,-0.002895,0.000818,...,-0.003095,-0.002181,-0.004514,-0.003095,0.004094,-0.003460,-0.015191,-0.002686,-0.008855,-0.015842
Arugula,0.006201,-0.009178,-0.008720,-0.015683,-0.003291,-0.011068,-0.002351,1.000000,-0.004354,-0.008977,...,-0.004655,-0.019580,-0.006790,-0.004655,-0.014979,-0.005205,-0.010282,-0.005739,0.022768,0.000328
Asian Pear,-0.003925,-0.002026,-0.001925,0.052659,-0.000726,-0.002443,-0.002895,-0.004354,1.000000,-0.003802,...,-0.001028,-0.004322,-0.001499,-0.001028,-0.003307,-0.001149,-0.005044,-0.004416,0.010414,-0.005260
Asparagus,-0.015525,0.026885,-0.007614,-0.019344,-0.002873,-0.009664,0.000818,-0.008977,-0.003802,1.000000,...,-0.004065,0.007810,-0.005929,-0.004065,-0.002313,0.026133,-0.019952,0.023192,0.001908,-0.020807
